In [1]:
# !pip install Praw
# !pip install --upgrade praw

In [2]:
#import sys
#!{sys.executable} -m pip install praw
import praw
import pandas as pd
import datetime as dt

f = open("secret_python.txt", "r")
client_id = f.readline().rstrip()
secret_key = f.readline().rstrip()
f.close()

In [3]:
reddit = praw.Reddit(client_id=client_id, \
                     client_secret=secret_key, \
                     user_agent='Scrape')

reddit.read_only

True

In [4]:
me_irl = reddit.subreddit('me_irl')

In [5]:
top_me_irl = me_irl.top(limit=20)

In [6]:
for submission in top_me_irl:
    print(submission.title, submission.id, submission.score)

Me📢irl cs507j 142930
me irl dy60tm 132349
me_irl bpdggk 131241
me irl elehnu 122347
me_irl djgfa6 121073
me irl ejytc4 119846
me_irl 5meh0q 109325
me irl 6r1ot2 108415
me irl ekv7it 106883
me_irl ecrjmx 103370
me🐱irl fwsnwz 101671
me_irl 65fni1 100361
me irl ecyv14 99250
Me_irl el8p5n 97667
me irl 8vzmij 95905
me_irl ap8pfl 95298
me irl cz617v 90310
Me irl fkshys 89015
me irl 6mvi19 88523
me_irl 5ytzv9 87377
